<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/idash_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,092 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [865 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [823 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,231 kB]
Hit:12 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.c

In [122]:
## 무조건 해야하는 부분 2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

In [123]:
## 한개씩 클릭해서 크롤링 하려함 (구조가 통일된 현행법만 크롤링하기 위해)
from selenium.webdriver.common.by import By
import queue
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver.implicitly_wait(10)
driver.get("https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01")
# time.sleep(3)

driver.switch_to.frame('indexFrame') # html안에 html로 스위치
driver.find_element(By.CSS_SELECTOR, "#searchListForm > div.mainOption > label:nth-child(4)").click() # 이건 한국어, 영어 법령으로 보겠다는 의미
# time.sleep(1)

table_list = [f'#\\{i} > td:nth-child(2) > p > a' for i in range(31,40)] + [f'#\\31 {i} > td:nth-child(2) > p > a' for i in range(10)] + ['#\\32 0 > td:nth-child(2) > p > a']
pages = [f'#container > div > div.paging > a:nth-child({i})' for i in range(4,14)]
cur_sum_pages = 1
cur_law = 1
eng_queue = queue.Queue()
final_df = pd.DataFrame(columns=['eng','kor'])
while cur_sum_pages < 118:
  for p in pages:
    for ind in table_list:
      driver.find_element(By.CSS_SELECTOR, ind).click() # 여기가 이제 1번부터 클릭해서 새로운탭으로 들어가는 코드
      # time.sleep(3)

      driver.switch_to.window(driver.window_handles[-1]) # 새로 열린 탭으로 작업환경 전환
      # time.sleep(1)
      driver.switch_to.frame('lawViewContent') # html안에 html로 스위치, 즉 법령내용이 포함된 html로 접근 가능
      # time.sleep(1)
      req = driver.page_source
      soup = BeautifulSoup(req, 'html.parser')
      corpus = soup.find_all('span') # 내용만 긁자

      eng = ''
      kor = ''
      for i in corpus: # corpus에서 tag 하나씩 eng, kor 매칭해서 저장하기
        # if 'A'<= i.text.strip()[0] <= 'z' and len(i.text.strip()) != 0:
        clean_text = i.text.strip()
        # if len(clean_text) == 0 or clean_text[0] == '&' or clean_text[0] == '<':
        #   continue
        # elif len(re.sub('[^A-Za-z]','', clean_text)) > 0:
        #   eng_queue.put(clean_text)
        # elif eng_queue.qsize() > 0 and len(clean_text) != 1:
        #   eng = eng_queue.get()
        #   kor = clean_text
        #   final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'eng': eng, 'kor': kor}])])

        if clean_text == ';' or clean_text == '//':
          continue
        if len(re.sub('[^A-Za-z]','', clean_text)) > 0:
          if eng != '' and kor != '':
            final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'eng': eng, 'kor': kor}])])
            eng = clean_text
            kor = ''
          eng += clean_text
        else:
          kor += clean_text

      cur_law += 1
      driver.close() # 현재 탭 닫기
      driver.switch_to.window(driver.window_handles[0]) # 원래 탭으로 전환, 즉 법령 list가 있는 탭
      driver.switch_to.frame('indexFrame') # html안에 html로 스위치
      # time.sleep(3)
      print(f'a number of crawled laws : {cur_law}')
    driver.find_element(By.CSS_SELECTOR, p).click() # 테이블에 있는 모든 법령을 크롤링 한 뒤 다음 페이지로 이동
    # time.sleep(2)
    cur_sum_pages += 1
    print(f'a number of crawled pages : {cur_sum_pages}')
    break
  break

a number of crawled laws : 2
a number of crawled laws : 3
a number of crawled laws : 4
a number of crawled laws : 5


KeyboardInterrupt: ignored

In [95]:
corpus2 = soup.find_all('td')
corpus2[12].text.strip()

'The master plans for rescue and emergency medical services (hereinafter referred to as “master plan”) under Article 6 (1) of the Act on 119 Rescue and Emergency Medical Services (hereinafter referred to as the “Act”) shall be formulated in consultation with the central rescue and emergency medical services policy council (hereinafter referred to as “central policy council”) under Article 27 (1) of the Act every five years.'

In [103]:
corpus2[21].text.strip()

'「119구조ㆍ구급에 관한 법률」(이하 “법”이라 한다) 제6조제1항에 따른 구조ㆍ구급 기본계획(이하 “기본계획”이라 한다)은 법 제27조제1항에 따른 중앙 구조ㆍ구급정책협의회(이하 “중앙 정책협의회”라 한다)의 협의를 거쳐 5년마다 수립하여야 한다.'

In [127]:
final_df = final_df.reset_index(drop=True)
final_df.iloc[5].kor

'소방청장은 제3조의 업무를 수행하기 위하여 관계 중앙행정기관의 장과 협의하여 대통령령으로 정하는 바에 따라 구조ㆍ구급 기본계획(이하 “기본계획”이라 한다)을 수립ㆍ시행하여야 한다.<개정 2014. 11. 19., 2017. 7. 26.>기본계획에는 다음 각 호의 사항이 포함되어야 한다.구조ㆍ구급서비스의 질 향상을 위한 정책의 기본방향에 관한 사항구조ㆍ구급에 필요한 체계의 구축, 기술의 연구개발 및 보급에 관한 사항구조ㆍ구급에 필요한 장비의 구비에 관한 사항구조ㆍ구급 전문인력 양성에 관한 사항구조ㆍ구급활동에 필요한 기반조성에 관한 사항구조ㆍ구급의 교육과 홍보에 관한 사항그 밖에 구조ㆍ구급업무의 효율적 수행을 위하여 필요한 사항소방청장은 기본계획에 따라 매년 연도별 구조ㆍ구급 집행계획(이하 “집행계획”이라 한다)을 수립ㆍ시행하여야 한다.<개정 2014. 11. 19., 2017. 7. 26.>소방청장은 제1항 및 제3항에 따라 수립된 기본계획 및 집행계획을 관계 중앙행정기관의 장, 특별시장ㆍ광역시장ㆍ특별자치시장ㆍ도지사ㆍ특별자치도지사(이하 “시ㆍ도지사”라 한다)에게 통보하고 국회 소관 상임위원회에 제출하여야 한다.<개정 2012. 3. 21., 2014. 11. 19., 2017. 7. 26.>소방청장은 기본계획 및 집행계획을 수립하기 위하여 필요한 경우에는 관계 중앙행정기관의 장 또는 시ㆍ도지사에게 관련 자료의 제출을 요청할 수 있다. 이 경우 자료제출을 요청받은 관계 중앙행정기관의 장 또는 시ㆍ도지사는 특별한 사유가 없으면 이에 따라야 한다.<개정 2014. 11. 19., 2017. 7. 26.>'

In [3]:
## 인터넷 도메인으로 숫자 바꿔가면서 크롤링 하려고 함
driver.get("https://elaw.klri.re.kr/kor_service/lawView.do?hseq=10020&lang=KOR")
driver.switch_to.frame('lawViewContent') # html안에 html로 스위치

req = driver.page_source
soup = BeautifulSoup(req, 'html.parser')
contents = soup.find_all('span') # 내용만 긁자
contents

[<span>The purpose of this Decree is to prescribe the matters which are delegated by the </span>,
 <span> (hereinafter referred to as the “Act”) and those which are necessary for the enforcement of the Act.</span>,
 <span>이 영은 지적법(이하 "법"이라 한다)에서 위임된 사항과 그 시행에 관하여 필요한 사항을 규정함을 목적으로 한다.</span>,
 <span>Besides the terms defined in the Act, for the purpose of this Decree, </span>,
 <span class="revision">&lt;Amended by Presidential Decree No. 14568, Apr. 6, 1995&gt;</span>,
 <span> The term “book” means a terrier and a forestry cadastral book;</span>,
 <span> The term “map” means a land cadastral map and a forestry cadastral map;</span>,
 <span> The term “duplicate” means the duplicate of a terrier, a land cadastral map, a forestry cadastral book and a forestry cadastral map respectively;</span>,
 <span> The term “surveying results” means the result of surveying which is put on record in a surveying book, a surveying result map and an area measurement book, which are made out after cadastr

In [46]:
text_list[2:25]

[<span>The terms used in this Act are defined as follows: &lt;Amended on Jan. 27, 2016; Oct. 20, 2020; Jan. 5, 2021&gt;</span>,
 <span> The term "rescue" means all operations performed to protect life, body, and property of a person who needs outside help (hereinafter referred to as "person in need of rescue") in the cases of fire, disaster, accident or terrorism, and other emergencies (hereinafter referred to as "emergency");</span>,
 <span> The term "119 rescue squad" means a unit organization comprised of firefighting officials and provided with equipment necessary for search and rescue operations;</span>,
 <span> The term "emergency medical services" means services, including consultation, first aid, and transportation provided to emergency patients;</span>,
 <span> The term "119 EMS (emergency medical services) team" means a unit organization comprised of firefighting officials with equipment necessary for emergency medical services;</span>,
 <span> The term "emergency patient" me

In [62]:


eng_queue = queue.Queue()

for i in text_list:
  if 'A'<= i.text[0] <= 'z':
    eng_queue.put(i.text)
  else:
    eng = eng_queue.get()
    kor = i.text
    print(eng)
    print(kor)
    break

The purpose of this Act is to strengthen the national capacity to rescue and provide emergency medical services, protect the lives, bodies, and property of the people, and contribute to the improvement of the quality of lives of the people by prescribing necessary matters concerning the efficient operation of 119 rescue and emergency medical services in the cases of fire, disaster, accident, terrorism, or other emergencies.
이 법은 화재, 재난ㆍ재해 및 테러, 그 밖의 위급한 상황에서 119구조ㆍ구급의 효율적 운영에 관하여 필요한 사항을 규정함으로써 국가의 구조ㆍ구급 업무 역량을 강화하고 국민의 생명ㆍ신체 및 재산을 보호하며 삶의 질 향상에 이바지함을 목적으로 한다.


In [52]:
queue.get()

1

In [57]:
queue.qsize()

0

In [ ]:
# json 만들어보자
data = []

count = 1
len = 0
tor = 0
eng = True
while tor > 10000:
  driver.get(f"https://elaw.klri.re.kr/kor_service/lawView.do?hseq={count}&lang=KOR")
  driver.switch_to.frame('lawViewContent') # html안에 html로 스위치

  req = driver.page_source
  soup = BeautifulSoup(req, 'html.parser')
  text_list = soup.find_all('span') # 내용만 긁자

  for i in text_list:
    if 'A'<= i.text[0] <= 'z':
      len += 1
      data.append({'en':i.text})
    elif len > 0: